# AI Learns to play Dino game!

In [2]:
# Importing Libraries

# Bread and butter
import numpy as np
import matplotlib.pyplot as plt

# Importing Selenium to control the browser
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

# To use our keyboard
import pyautogui

# TO help me become Doctor Strange
import time

# Because I can't do Image processing myself
import cv2 

# Deep Learning Libraries
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
# Pur environment class which is used to play the game
class Environment():

    def __init__(self, device="cpu", chrome_path=".//Driver/chromedriver.exe"):
        self.done = False
        self.device = device
        self.initialize_chrome(chrome_path)
        
    def initialize_chrome(self, chrome_path):
        chrome_options = Options()
        chrome_options.add_argument("start-maximized")
        chrome_options.add_argument("disable-infobars")
        chrome_options.add_argument("--mute-audio")
        self.driver = webdriver.Chrome(chrome_path, chrome_options=chrome_options)
        
    def reset(self, game_url="chrome://dino"):
        try:
            self.driver.get(game_url)
        except:
            pass
    
    def start(self):
        pyautogui.press("up")

    # Hard stop
    def close_all(self):
        self.driver.close()
        self.driver.quit()
        try:
            os.system("cmd /c taskkill /F /IM chromedriver.exe")
        except:
            print("No tasks found!")
    

    def get_screenshot(self, screenshot_path=".//screenshot/"):
        current_state = []
        for i in range(1, 5):
            file_name = screenshot_path + str(i) + '.jpg'
            screenshot = pyautogui.screenshot(region=(0, 400, 1920, 330))
            screenshot.save(file_name)
            image_tensor = self.process_image(file_name)
            current_state.append(image_tensor)
        current_state = torch.cat(current_state).unsqueeze(0).to(self.device)

    def process_image(self, image_file):
        image = cv2.imread(image_file)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image[image>255] = 255
        image = cv2.resize(image, (256, 64))
        image = np.reshape(image, (1, 256, 128))
        return self.image_to_tensor(image)

    def image_to_tensor(self, image):
        image = image.astype(np.float32)
        image_tensor = torch.from_numpy(image)
        image_tensor = image_tensor.to(self.device, dtype=torch.float32)
        return image_tensor


    def jump(self):
        pyautogui.keyDown("up")
        time.sleep(0.2)
        pyautogui.keyUp("up")


    def duck(self):
        pyautogui.keyDown("down")
        time.sleep(0.25)
        pyautogui.keyUp("down")


    def do_nothing(self):
        time.sleep(0.25)


    def take_action(self, action):
        reward = 1
        if action == 0:
            self.jump()
        elif action == 1:
            self.duck()
        elif action == 2:
            self.do_nothing()
        
        state = self.get_screenshot()

        
        

In [84]:
env = Environment()
env.reset()
env.start()
time.sleep(2)
env.jump()
time.sleep(2)
env.duck()
time.sleep(2)
env.close_all()

In [72]:
env.jump()